# 2pt

In [49]:
import h5py,os,re,click
import numpy as np
from itertools import permutations
from sympy.combinatorics import Permutation

ens='cE211.044.112'

path='data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

#!============== mom opearations ==============#
if True:
    def mom2msq(mom):
        assert(len(mom)==3)
        return mom[0]**2+mom[1]**2+mom[2]**2
    def getSortkey_mom(mom): # @mom2standard
        msq=mom2msq(mom)
        return (msq,-mom[2],-mom[1],-mom[0])
    def mom2standard(mom): # @getSortkey_mom
        return sorted([abs(mom[0]),abs(mom[1]),abs(mom[2])]) 
    
    elements_rot48=[(sx,sy,sz,xyz) for sx in [1,-1] for sy in [1,-1] for sz in [1,-1] for xyz in [(0,1,2),(0,2,1),(1,0,2),(1,2,0),(2,0,1),(2,1,0)]]
    def rotate_mom(e,mom):
        if len(mom)!=3:
            return rotate_mom(e,mom[:3])+rotate_mom(e,mom[3:])
        sx,sy,sz,xyz=e; ix,iy,iz=xyz; iix,iiy,iiz=tuple([ix,iy,iz].index(i) for i in range(3))
        return [sx*mom[iix],sy*mom[iiy],sz*mom[iiz]]
    def mom2moms(mom):
        return [list(mom) for mom in {tuple(rotate_mom(e,mom)) for e in elements_rot48}] 
    
max_mom2=64
range_xyz=range(-int(np.sqrt(max_mom2))-1,int(np.sqrt(max_mom2))+2)
moms=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2]       
moms=list(set([tuple(mom2standard(mom)) for mom in moms]))
moms=[list(mom) for mom in moms]; 
moms=sorted(moms,key=getSortkey_mom)
moms_target=moms

def extractFile(paths):
    Nsrcs=[]
    for ipath,path in enumerate(paths):
        with h5py.File(path) as f:
            Nsrcs.append(len(f['srcs']))
    
    dat=0
    for ipath,path in enumerate(paths):
        with h5py.File(path) as f:
            t1=f['data/N1_N1'][:]; t2=f['data/N2_N2'][:]
            t=(t1+t2)/2
            t=np.mean(t,axis=-1)

            t1=f['data_bw/N1_N1'][:]; t2=f['data_bw/N2_N2'][:]
            t_bw=(t1+t2)/2
            t_bw=np.mean(t_bw,axis=-1)
            
            t_bw=np.flip(t_bw,axis=0)
            # t_bw=t_bw[:,inds_negmom]
            t[1:] = (t[1:] + (-1)*t_bw[:])/2
            
            moms=f['moms'][:]
            dic={}
            for i,m in enumerate(moms):
                dic[tuple(m)]=i

            t=np.transpose([np.mean(t[:,[dic[tuple(m)] for m in mom2moms(mom)]],axis=1) for mom in moms_target],[1,0])
            
            weight=Nsrcs[ipath]/np.sum(Nsrcs)
            dat += np.real(t*weight)
    return dat
    
data=[]
for icfg,cfg in enumerate(cfgs[410:411]):
    print(f'{icfg}/{len(cfgs)}',end='        \r')
    
    path=f'/p/project1/ngff/li47/code/projectData/05_moments/{ens}/data_N_fullmom/{cfg}/'
    data.append(extractFile([f'{path}/{file}' for file in os.listdir(path) if ('run4' not in file) and ('run5' not in file)]))

data=np.array(data)

# path=f'/p/project1/ngff/li47/code/projectData/05_moments/{ens}/data_merge/disc_2pt.h5'
# with h5py.File(path,'w') as f:
#     f.create_dataset('notes',data=['cfg,time,mom'])
#     f.create_dataset('cfgs',data=cfgs)
#     f.create_dataset('moms',data=moms_target)
    
#     f.create_dataset('data/N_N',data=data)

print(data.shape)

(1, 57, 80)  


/tmp/ipykernel_1105020/1088638263.py:50: RuntimeWarning: invalid value encountered in divide
  t=(t1+t2)/2


In [75]:
path='/p/project1/ngff/li47/code/projectData/05_moments/cE211.044.112/data_N_fullmom/d0352/N.h5_Nrun2_127'
with h5py.File(path) as f:
    t=f['data/N2_N2']
    print(t.shape)
    print(t[:16,0,0])
    print(t[6,0,0])

(57, 2109, 2)
[ 1.0175419e-10-5.1544950e-13j  2.5773803e-10-1.0312250e-12j
  1.6366851e-10-1.3036139e-12j  1.0939998e-10+1.4737686e-12j
  4.2120382e-06+1.7305374e+00j            inf          +nanj
            inf          +nanj  5.4853951e+30+6.2110817e-01j
  2.3872912e-11-1.7985359e+00j -1.1285164e+22+4.4034547e-01j
  9.1660281e+35+6.5844530e-01j  2.4737799e-11+1.7469832e+00j
  8.5907576e+15-1.4577904e+00j -4.8055490e+14+1.8628734e+00j
 -3.5569411e+30-4.0908760e-01j -4.3823585e+09+1.6170362e+00j]
(inf+nanj)


In [85]:
import tarfile
path='/p/project1/ngff/li47/code/projectData/05_moments/cE211.044.112/data_pre/Nrun2_127_arch/0352_r3.tar'
with tarfile.open(path) as tar:
    for member in tar.getmembers():
        if not member.name.endswith('.h5'):
            continue
        if not ('twop' in member.name):
            continue
        if not ('sx79sy20sz87st200' in member.name):
            continue
        fileobj = tar.extractfile(member)
        with h5py.File(fileobj) as f:
            srcs_old=list(f.keys())
            src_old=srcs_old[0]
            t=f[src_old]['baryons/nucl_nucl/twop_baryon_2']
            print(src_old,t[6,0,0,0])
            t=f[src_old]['baryons/nucl_nucl/twop_baryon_1']
            print(src_old,t[6,0,0,0])
        # break

sx79sy20sz87st200 inf
sx79sy20sz87st200 0.0


# jvev

In [ ]:
import h5py,os,re,click
import numpy as np
from itertools import permutations
from sympy.combinatorics import Permutation

ens='cB211.072.64'

jqs=['j+','js','jc'] # disc
stouts=range(40+1) # gluon

path='data_aux/cfgs_run'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
inds_trace=[i for i,ins in enumerate(inserts) if ins[0]==ins[1]]

xyzt2xyz0=lambda x: x if x!='t' else '0'
t=[insert for insert in inserts]; inserts_key=[f'=der:g{xyzt2xyz0(insert[1])}D{xyzt2xyz0(insert[0])}:sym=' for insert in t]

def extractVEV(basepath):
    j2data={}
    
    txyz=['t','x','y','z']
    Dmus=['d3','d0','d1','d2']
    gnus=['gt','gx','gy','gz']
    
    path=f'{basepath}/j.h5'
    with h5py.File(path) as f:
        for j in jqs:
            gms=[gm.decode() for gm in f['inserts'][:]]
            moms_old=f['moms'][:]
            dic={}
            for i,m in enumerate(moms_old):
                dic[tuple(m)]=i
            imom=dic[(0,0,0)]
            
            t=np.array([[f[f'data/{j};{Dmu}'][:,:,gms.index(gnu)] for Dmu in Dmus] for gnu in gnus])
            t=t[:,:,:,imom]
            t=(t+np.transpose(t,[1,0,2]))/2
            t=t - np.eye(4)[:,:,None]*np.trace(t,axis1=0,axis2=1)[None,None,:]/4
            t=np.transpose([t[txyz.index(m),txyz.index(n)] for m,n in inserts],[1,0])
            
            j2data[f'{j};disc']=np.mean(t,axis=0)

    path=f'{basepath}/jg.h5'
    with h5py.File(path) as f:
        gms=[gm.decode() for gm in f['inserts'][:]]
        moms_old=f['moms'][:]
        dic={}
        for i,m in enumerate(moms_old):
            dic[tuple(m)]=i
        imom=dic[(0,0,0)]
        for stout in stouts:
            j=f'jg;stout{stout}'
            
            t=f[f'data/{j}'][:]
            t=t[:,imom]
            j2data[j]=np.mean(t,axis=0)
            
    return j2data

js=[f'{j};disc' for j in jqs] + [f'jg;stout{stout}' for stout in stouts]
j2data={j:[] for j in js}
for icfg,cfg in enumerate(cfgs):
    print(f'{icfg}/{len(cfgs)}',end='        \r')
    
    basepath=f'/p/project1/ngff/li47/code/projectData/02_discNJN_1D/{ens}/data_post_hold/{cfg}/'
    j2dat=extractVEV(basepath)
    
    for j in js:
        j2data[j].append(j2dat[j])
    # break

path=f'/p/project1/ngff/li47/code/projectData/05_moments/{ens}/data_merge/disc_jvev.h5'
with h5py.File(path,'w') as f:
    for j in j2data.keys():
        f.create_dataset(j,data=j2data[j])